# Query Playground

Run direct FRAG workflow queries from a notebook and inspect answer metadata.

In [1]:
from pathlib import Path
import os
import sys

ROOT = Path.cwd().resolve()
if (ROOT / 'backend').exists():
    PROJECT_ROOT = ROOT
elif ROOT.name == 'notebook':
    PROJECT_ROOT = ROOT.parent
else:
    PROJECT_ROOT = ROOT

BACKEND_DIR = PROJECT_ROOT / 'backend'
if str(BACKEND_DIR) not in sys.path:
    sys.path.insert(0, str(BACKEND_DIR))

os.chdir(BACKEND_DIR)

In [2]:
import pandas as pd
from app.core.database import db_manager
from app.workflows.enhanced_frag import process_enhanced_query

await db_manager.init_postgres()
db_manager.init_neo4j()

C:\Users\balas\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Researchaiagent\backend\app\services\gemini_speculative_rag.py:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


✓ Gemini API initialized and tested successfully with gemini-3-flash-preview


In [3]:
QUERY = 'Summarize the main findings from the uploaded documents.'
DOCUMENT_IDS = None  # Example: [1, 2, 3]

response = await process_enhanced_query(QUERY, document_ids=DOCUMENT_IDS)
print('Answer:')
print(response.answer)
print('\nConfidence:', response.confidence)
print('Query type:', response.query_type)


=== ENHANCED QUERY PROCESSING ===
Query: Summarize the main findings from the uploaded documents.
Document IDs filter: None
Retrieved 15 chunks from database
→ Generating answer with Gemini...

=== GENERATION START ===
Query: Summarize the main findings from the uploaded documents.
Chunks available: 15
Gemini available: True
→ Attempting Gemini generation...
✓ Gemini generation successful
✓ Answer generated successfully with model: gemini-3-flash-preview
Answer:
Based on the provided documents, the main findings cover graph database benchmarking, data modeling comparisons, industry research needs, and emerging technologies in AI and data storage.

### **Graph Database Benchmarking and Performance**
The Linked Data Benchmark Council (LDBC) benchmarks are identified as the most widely adopted standards in the industry [Source 3]. These benchmarks include:
*   **LDBC-SNB (Social Network Benchmark):** Used for interactive workloads (graph OLTP) and business intelligence workloads (relatio

In [4]:
sources_df = pd.DataFrame(response.sources) if response.sources else pd.DataFrame()
sources_df

,chunk_id,document_id,content_preview
0,157,42,Graph Technology Landscape 2020. https://graph...
1,64,40,•Participated in full software development lif...
2,136,42,"No- tably, LDBC benchmarks are the most widely..."
3,146,42,8.5 Recommendation for Researchers The graph d...
4,153,42,"Cochez, C. D’amato, G. D. Melo, C. Gutierrez, ..."
5,169,43,Here the reconstructed data page propagates ba...
6,114,42,Fig- ure 1(b) shows how the same information c...
7,154,42,"Chen, and S. Salihoglu. Graphﬂow: An active gr..."


In [5]:
await db_manager.close()